In [1]:
import numpy as np
import copy
from scipy.sparse import csr_matrix

In [2]:
def isvalidmove(s, istart, iend):
    if len(s[istart]) == 0: return False
    if len(s[iend]) == 0: return True
    if s[istart][-1]<s[iend][-1]: 
        return True
    else:
        return False

In [3]:
def generateHanoi3status_old(n):
    res = {}
    #res[0] = [[],[],[3,2,1]]
    res[0] = [[3,2,1],[],[]]
    for i in range(1,n):
        for start, end in [(0,1), (1,0), (1,2), (2,1)]:
            if isvalidmove(res[i-1], start, end):
                temp = copy.deepcopy(res[i-1])
                temp[end].append(temp[start].pop()) 
                if temp not in res.values():
                    res[i] = temp
                    continue
    return res

In [4]:
def generateHanoi3status(n):
    res = {}
    valid_move = {i: [] for i in range(n)}
    res[0] = ([3,2,1],[],[])
    i = 1
    layer = [0]
    while i<n and len(layer) != 0:
        next_layer = []
        for curr_idx in layer:
            curr_status = res[curr_idx]
            for start, end in [(0,1),(1,0),(1,2),(2,1)]:
                if isvalidmove(curr_status, start, end):
                    temp = copy.deepcopy(curr_status)
                    temp[end].append(temp[start].pop()) 
                    if temp not in res.values():
                        res[i] = temp
                        next_layer.append(i)
                        valid_move[curr_idx].append(i)
                        valid_move[i].append(curr_idx)
                        i += 1
        layer = next_layer
    return [res, valid_move]

In [5]:
def computecenterHanoi3(s):
    weightedsum = 0.0
    for i in range(3):
        weightedsum += sum(s[i]) * i
    return weightedsum / 6.0

In [6]:
def computeprobHanoi3(n,T,valid_move):
    i, j, v = [], [], []
    for ii in range(n):
        for jj in valid_move[ii]:
            i.append(jj)
            j.append(ii)
            v.append(1.0/float(len(valid_move[ii])))
    i, j, v = np.array(i), np.array(j), np.array(v)
#    i = np.concatenate((np.arange(1,n),np.arange(0,n-1)), axis=0)
#    j = np.concatenate((np.arange(0,n-1),np.arange(1,n)), axis=0)
#    v = np.concatenate((np.array([1.0]),[0.5]*(2*n-4),np.array([1.0])), axis=0)
    A = csr_matrix((v,(i,j)),shape=(n,n))
    #res = np.concatenate((np.array([1.0]),[0]*(n-1)), axis=0)
    res = csr_matrix(([1],([0],[0])),shape=(n,1))
    for k in range(T):
        res = A * res
    return res

In [7]:
def computeprobHanoi3_old(n,T):
    i = np.concatenate((np.arange(1,n),np.arange(0,n-1)), axis=0)
    j = np.concatenate((np.arange(0,n-1),np.arange(1,n)), axis=0)
    v = np.concatenate((np.array([1.0]),[0.5]*(2*n-4),np.array([1.0])), axis=0)
    A = csr_matrix((v,(i,j)),shape=(n,n))
    #res = np.concatenate((np.array([1.0]),[0]*(n-1)), axis=0)
    res = csr_matrix(([1],([0],[0])),shape=(n,1))
    for k in range(T):
        res = A * res
    return res

In [10]:
# Main　Hanoi3
n = 27
T = 16
status, valid_move = generateHanoi3status(n)
#status = generateHanoi3status_old(n)
center = [[computecenterHanoi3(status[i]) for i in range(n)]]
#p = computeprobHanoi3_old(n,T).todense()
p = computeprobHanoi3(n,T,valid_move).todense()
mean = np.dot(center, p)
variance = np.dot(np.square(center-mean), p)
print("%.10f" % mean)
print("%.10f" % variance)
#for i in range(n):
#    print(i, status[i], center[i])

0.4094034831
0.0806662403


In [9]:
#print(p)